<a href="https://colab.research.google.com/github/ppthaw2024/Kristal-Transaction-Monitoring/blob/main/CRS_Automation_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#Not counting the Number of Joint Holders

import pandas as pd
import numpy as np

# ---------- Paths ----------
raw_path = '/content/crs_export_KASG.xlsx'
template_path = '/content/KAMEL - CRS Submission.xlsx'

# ---------- Load data ----------
raw_data = pd.read_excel(raw_path, sheet_name=0)
# If your template has a title row, header=1 captures the actual headers
template  = pd.read_excel(template_path, sheet_name='Accounts', header=1)
required_columns = list(template.columns)

# ---------- Helpers ----------
def safe_series(df, col, default=""):
    return df[col] if col in df.columns else pd.Series([default] * len(df), index=df.index)

def concat_with_sep(sep, *series_list):
    parts = []
    for s in series_list:
        s_clean = s.astype(str).str.strip().replace({"nan": "", "None": ""})
        s_clean = s_clean.mask(s_clean.eq(""))  # empty -> NaN
        parts.append(s_clean)
    stacked = pd.concat(parts, axis=1)
    return stacked.apply(lambda r: sep.join([x for x in r.dropna().astype(str)]), axis=1).fillna("")

def sum_numeric(*series_list):
    total = pd.Series(0.0, index=series_list[0].index, dtype="float64")
    for s in series_list:
        total = total.add(pd.to_numeric(s, errors="coerce").fillna(0.0), fill_value=0.0)
    return total

def parse_date(series, fmt_out="%d/%m/%Y"):
    parsed = pd.to_datetime(series, errors="coerce")
    out = pd.Series("", index=series.index, dtype="object")
    mask = parsed.notna()
    out.loc[mask] = parsed.loc[mask].dt.strftime(fmt_out)
    out.loc[~mask] = series.astype(str).where(~series.isna(), "")
    return out

def clean_tin_preserve(series: pd.Series) -> pd.Series:
    """
    Preserve TIN exactly unless it is empty/NaN or all zeros -> '0'.
    Handles numeric-looking inputs like 12345.0 -> '12345'.
    """
    def norm(v):
        if pd.isna(v):
            return "0"
        if isinstance(v, (int, np.integer)):
            s = str(v)
        elif isinstance(v, float):
            if np.isfinite(v) and v.is_integer():
                s = str(int(v))
            else:
                s = str(v)
        else:
            s = str(v)
        s = s.strip()
        if s == "" or s.lower() in ("nan", "none"):
            return "0"
        # zeros-only string -> "0"
        if len(s) > 0 and set(s) <= {"0"}:
            return "0"
        return s
    return series.apply(norm).astype(str)

# ---------- Mapping spec ----------
# kinds: "copy", "concat", "const", "date", "sum", "tin_keep"
mapping_spec = {
    "Number": ("copy", ["application_id"]),
    "Holder Individual": ("copy", ["client_id"]),
    "Holder Entity": ("copy", ["controlling_client_id"]),

    "First Name": ("copy", ["first_name"]),
    "Last Name":  ("copy", ["last_name"]),

    # Street = address_1 + address_2 with comma only when both exist
    "Street": ("concat", ["residential_address_1", "residential_address_2"], {"sep": ", "}),

    "City": ("copy", ["residential_city"]),
    "Country": ("copy", ["country_of_residence"]),
    "Postal Code": ("copy", ["residential_postal_code"]),

    # TIN rule: preserve exactly unless empty/NaN/zeros-only -> '0'
    "TIN 1": ("tin_keep", ["primary_tax_id"]),
    "TIN Issuer 1": ("copy", ["primary_tax_country"]),
    "Tax Residency 1": ("copy", ["primary_tax_country"]),

    # DOB -> dd/mm/YYYY
    "Date of Birth": ("date", ["date_of_birth"], {"fmt": "%d/%m/%Y"}),

    # Business rules
    "Currency": ("const", "USD"),
    "Type": ("copy", ["booking_center"]),
    "Number Type": ("copy", ["client_type"]),

    # Choose one for Balance:
    "Balance": ("copy", ["cash_balance"]),
    # Or: "Balance": ("sum", ["application_aum", "cash_balance"]),
}

# ---------- Build submission in template order ----------
submission_df = pd.DataFrame(index=raw_data.index)

for tgt in required_columns:
    spec = mapping_spec.get(tgt)
    if not spec:
        submission_df[tgt] = ""  # unmapped targets left blank
        continue

    kind = spec[0]
    sources = spec[1] if len(spec) > 1 else []
    opts = spec[2] if len(spec) > 2 else {}

    if kind == "copy":
        submission_df[tgt] = safe_series(raw_data, sources[0]).astype(str).str.strip()

    elif kind == "concat":
        series_list = [safe_series(raw_data, s).astype(str) for s in sources]
        submission_df[tgt] = concat_with_sep(opts.get("sep", " "), *series_list)

    elif kind == "const":
        value = spec[1] if isinstance(spec[1], str) else opts
        submission_df[tgt] = pd.Series([value]*len(raw_data), index=raw_data.index)

    elif kind == "date":
        fmt = opts.get("fmt", "%d/%m/%Y")
        submission_df[tgt] = parse_date(safe_series(raw_data, sources[0]), fmt_out=fmt)

    elif kind == "sum":
        series_list = [safe_series(raw_data, s, default=0) for s in sources]
        submission_df[tgt] = sum_numeric(*series_list).round(2)

    elif kind == "tin_keep":
        submission_df[tgt] = clean_tin_preserve(safe_series(raw_data, sources[0]))

    else:
        submission_df[tgt] = ""

# Keep exact template order
submission_df = submission_df.reindex(columns=required_columns)

# ---------- Entity name rule: move to "Name" instead of First/Last ----------
# If raw Last Name equals one of these tokens (case-insensitive), treat as entity.
ENTITY_LASTNAME_TOKENS = {
    "PTE LTD", "SN ISSUER AND BROKER", "LIMITED", "FUND MANAGER", "ISSUER",
    "CMA ACC PROVIDER", "COUNTERPARTY", "PVT MKT COUNTERPARTY", "CLIENT",
    "HOLDING LIMITED", "INTERNATIONAL TRUST"
}

# Build a mask from RAW last_name (not the mapped one) for strict equality with the tokens.
last_raw_norm = safe_series(raw_data, "last_name").astype(str).str.strip().str.upper()
entity_mask = last_raw_norm.isin(ENTITY_LASTNAME_TOKENS)

# If "Name" exists in template, set Name = "First Name + Last Name" (only for entity rows)
if "Name" in submission_df.columns:
    fn = submission_df.get("First Name", pd.Series("", index=submission_df.index)).astype(str)
    ln = submission_df.get("Last Name",  pd.Series("", index=submission_df.index)).astype(str)
    full_name = concat_with_sep(" ", fn, ln)
    # populate Name only where entity_mask is True
    submission_df.loc[entity_mask, "Name"] = full_name.loc[entity_mask]

# Blank out First/Last where entity_mask is True (since we moved it to Name)
if "First Name" in submission_df.columns:
    submission_df.loc[entity_mask, "First Name"] = ""
if "Last Name" in submission_df.columns:
    submission_df.loc[entity_mask, "Last Name"] = ""

# ---------- Optional clean-ups ----------
# Postal code: numeric-only and zero-pad to 6 (SG style)
if "Postal Code" in submission_df.columns:
    submission_df["Postal Code"] = (
        submission_df["Postal Code"].astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(6)
    )

# Is Joint? based on Number starting with J (only if present in template)
if "Is Joint?" in submission_df.columns:
    number_src = submission_df.get("Number", pd.Series("", index=submission_df.index)).astype(str).str.strip().str.upper()
    submission_df["Is Joint?"] = np.where(number_src.str.startswith("J"), "T", "F")

# ---------- Save ----------
out_path = 'submission_ready.xlsx'
submission_df.to_excel(out_path, index=False)
print(f"Saved: {out_path}")




Saved: submission_ready.xlsx


In [23]:
#Counting the number of Joint Holders. Should be lesser Rows
import pandas as pd
import numpy as np

# ---------- Paths ----------
raw_path = '/content/crs_export_KASG.xlsx'
template_path = '/content/KAMEL - CRS Submission.xlsx'

# ---------- Load data ----------
raw_data = pd.read_excel(raw_path, sheet_name=0)
# If the template has a title row, header=1 captures the actual headers
template  = pd.read_excel(template_path, sheet_name='Accounts', header=1)
required_columns = list(template.columns)

# ---------- Helpers ----------
def safe_series(df, col, default=""):
    return df[col] if col in df.columns else pd.Series([default] * len(df), index=df.index)

def concat_with_sep(sep, *series_list):
    parts = []
    for s in series_list:
        s_clean = s.astype(str).str.strip().replace({"nan": "", "None": ""})
        s_clean = s_clean.mask(s_clean.eq(""))  # empty -> NaN
        parts.append(s_clean)
    stacked = pd.concat(parts, axis=1)
    return stacked.apply(lambda r: sep.join([x for x in r.dropna().astype(str)]), axis=1).fillna("")

def sum_numeric(*series_list):
    total = pd.Series(0.0, index=series_list[0].index, dtype="float64")
    for s in series_list:
        total = total.add(pd.to_numeric(s, errors="coerce").fillna(0.0), fill_value=0.0)
    return total

def parse_date(series, fmt_out="%d/%m/%Y"):
    parsed = pd.to_datetime(series, errors="coerce")
    out = pd.Series("", index=series.index, dtype="object")
    mask = parsed.notna()
    out.loc[mask] = parsed.loc[mask].dt.strftime(fmt_out)
    out.loc[~mask] = series.astype(str).where(~series.isna(), "")
    return out

def clean_tin_preserve(series: pd.Series) -> pd.Series:
    """
    Preserve TIN exactly unless it is empty/NaN or all zeros -> '0'.
    Handles numeric-looking inputs like 12345.0 -> '12345'.
    """
    def norm(v):
        if pd.isna(v):
            return "0"
        if isinstance(v, (int, np.integer)):
            s = str(v)
        elif isinstance(v, float):
            if np.isfinite(v) and v.is_integer():
                s = str(int(v))
            else:
                s = str(v)
        else:
            s = str(v)
        s = s.strip()
        if s == "" or s.lower() in ("nan", "none"):
            return "0"
        if len(s) > 0 and set(s) <= {"0"}:  # zeros-only
            return "0"
        return s
    return series.apply(norm).astype(str)

# ---------- Mapping spec ----------
# kinds: "copy", "concat", "const", "date", "sum", "tin_keep"
mapping_spec = {
    "Number": ("copy", ["application_id"]),
    "Holder Individual": ("copy", ["client_id"]),
    "Holder Entity": ("copy", ["controlling_client_id"]),

    "First Name": ("copy", ["first_name"]),
    "Last Name":  ("copy", ["last_name"]),

    # Street = address_1 + address_2
    "Street": ("concat", ["residential_address_1", "residential_address_2"], {"sep": ", "}),

    "City": ("copy", ["residential_city"]),
    "Country": ("copy", ["country_of_residence"]),
    "Postal Code": ("copy", ["residential_postal_code"]),

    # TIN rule: preserve exactly unless empty/NaN/zeros-only -> '0'
    "TIN 1": ("tin_keep", ["primary_tax_id"]),
    "TIN Issuer 1": ("copy", ["primary_tax_country"]),
    "Tax Residency 1": ("copy", ["primary_tax_country"]),

    # DOB -> dd/mm/YYYY
    "Date of Birth": ("date", ["date_of_birth"], {"fmt": "%d/%m/%Y"}),

    # Business rules
    "Currency": ("const", "USD"),
    "Type": ("copy", ["booking_center"]),
    "Number Type": ("copy", ["client_type"]),

    # Balance rule (choose one)
    "Balance": ("copy", ["cash_balance"]),
    # Or: "Balance": ("sum", ["application_aum", "cash_balance"]),
}

# ---------- Build submission in template order ----------
submission_df = pd.DataFrame(index=raw_data.index)

for tgt in required_columns:
    spec = mapping_spec.get(tgt)
    if not spec:
        submission_df[tgt] = ""  # unmapped targets left blank
        continue

    kind = spec[0]
    sources = spec[1] if len(spec) > 1 else []
    opts = spec[2] if len(spec) > 2 else {}

    if kind == "copy":
        submission_df[tgt] = safe_series(raw_data, sources[0]).astype(str).str.strip()

    elif kind == "concat":
        series_list = [safe_series(raw_data, s).astype(str) for s in sources]
        submission_df[tgt] = concat_with_sep(opts.get("sep", " "), *series_list)

    elif kind == "const":
        value = spec[1] if isinstance(spec[1], str) else opts
        submission_df[tgt] = pd.Series([value]*len(raw_data), index=raw_data.index)

    elif kind == "date":
        fmt = opts.get("fmt", "%d/%m/%Y")
        submission_df[tgt] = parse_date(safe_series(raw_data, sources[0]), fmt_out=fmt)

    elif kind == "sum":
        series_list = [safe_series(raw_data, s, default=0) for s in sources]
        submission_df[tgt] = sum_numeric(*series_list).round(2)

    elif kind == "tin_keep":
        submission_df[tgt] = clean_tin_preserve(safe_series(raw_data, sources[0]))

    else:
        submission_df[tgt] = ""

# Keep exact template order
submission_df = submission_df.reindex(columns=required_columns)

# ---------- Entity name rule ----------
ENTITY_LASTNAME_TOKENS = {
    "PTE LTD", "SN ISSUER AND BROKER", "LIMITED", "FUND MANAGER", "ISSUER",
    "CMA ACC PROVIDER", "COUNTERPARTY", "PVT MKT COUNTERPARTY", "CLIENT",
    "HOLDING LIMITED", "INTERNATIONAL TRUST"
}
if "First Name" in submission_df.columns and "Last Name" in submission_df.columns:
    last_raw_norm = safe_series(raw_data, "last_name").astype(str).str.strip().str.upper()
    entity_mask = last_raw_norm.isin(ENTITY_LASTNAME_TOKENS)
    if "Name" in submission_df.columns:
        fn = submission_df.get("First Name", pd.Series("", index=submission_df.index)).astype(str)
        ln = submission_df.get("Last Name",  pd.Series("", index=submission_df.index)).astype(str)
        full_name = concat_with_sep(" ", fn, ln)
        submission_df.loc[entity_mask, "Name"] = full_name.loc[entity_mask]
    submission_df.loc[entity_mask, "First Name"] = ""
    submission_df.loc[entity_mask, "Last Name"]  = ""

# ---------- Joint holders counting (dtype-safe) & delete non-primary rows ----------
if "Number" in submission_df.columns:
    # Copy of original Number BEFORE filtering
    number_orig = submission_df["Number"].astype(str).str.strip()

    # Identify non-empty Numbers and prefixes
    num_nonempty = number_orig.where(number_orig.ne(""), pd.NA)
    starts_J = number_orig.str.upper().str.startswith("J")
    starts_S = number_orig.str.upper().str.startswith("S")

    # Count identical Number IDs (ignoring blanks)
    group_counts = num_nonempty.groupby(num_nonempty).transform("size")
    group_counts = pd.Series(group_counts, index=submission_df.index).astype("Int64")

    # Build an Int64 series: default 1, J* -> group size, S* -> NA (blank later)
    joint_vals = pd.Series(1, index=submission_df.index, dtype="Int64")
    joint_vals = joint_vals.mask(starts_J & num_nonempty.notna(), group_counts)
    joint_vals = joint_vals.mask(starts_S, pd.NA)

    # Decide the output column name
    joint_col = next((c for c in ["Nr Joint Holders", "Joint Holders"] if c in submission_df.columns),
                     "Nr Joint Holders")
    # Assign the Int64 series first
    submission_df[joint_col] = joint_vals

    # DELETE rows that are not primary_client_id
    if "Number Type" in submission_df.columns:
        keep_mask = submission_df["Number Type"].astype(str).str.strip().str.lower().eq("primary_client_id")
        submission_df = submission_df.loc[keep_mask].copy()

    # Convert <NA> to empty strings for Excel display
    submission_df[joint_col] = submission_df[joint_col].astype("object").where(
        submission_df[joint_col].notna(), ""
    )

# ---------- Optional: Is Joint? flag (if present) ----------
if "Is Joint?" in submission_df.columns:
    src_now = submission_df.get("Number", pd.Series("", index=submission_df.index)).astype(str)
    submission_df["Is Joint?"] = np.where(src_now.str.upper().str.startswith("J"), "T", "F")

# ---------- Optional clean-ups ----------
# SG postal code normalize
if "Postal Code" in submission_df.columns:
    submission_df["Postal Code"] = (
        submission_df["Postal Code"].astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(6)
    )

# ---------- Save ----------
out_path = 'submission_ready.xlsx'
submission_df.to_excel(out_path, index=False)
print(f"Saved: {out_path}")


Saved: submission_ready.xlsx


In [26]:
#adding more Last Name tokens (using REGEX) for KASG entity
import pandas as pd
import numpy as np
import re

# ---------- Paths ----------
raw_path = '/content/crs_export_KASG.xlsx'
template_path = '/content/KAMEL - CRS Submission.xlsx'

# ---------- Load data ----------
raw_data = pd.read_excel(raw_path, sheet_name=0)
# If the template has a title row above headers, header=1 captures the real headers
template  = pd.read_excel(template_path, sheet_name='Accounts', header=1)
required_columns = list(template.columns)

# ---------- Helpers ----------
def safe_series(df, col, default=""):
    return df[col] if col in df.columns else pd.Series([default] * len(df), index=df.index)

def concat_with_sep(sep, *series_list):
    parts = []
    for s in series_list:
        s_clean = s.astype(str).str.strip().replace({"nan": "", "None": ""})
        s_clean = s_clean.mask(s_clean.eq(""))  # empty -> NaN
        parts.append(s_clean)
    stacked = pd.concat(parts, axis=1)
    return stacked.apply(lambda r: sep.join([x for x in r.dropna().astype(str)]), axis=1).fillna("")

def sum_numeric(*series_list):
    total = pd.Series(0.0, index=series_list[0].index, dtype="float64")
    for s in series_list:
        total = total.add(pd.to_numeric(s, errors="coerce").fillna(0.0), fill_value=0.0)
    return total

def parse_date(series, fmt_out="%d/%m/%Y"):
    parsed = pd.to_datetime(series, errors="coerce")
    out = pd.Series("", index=series.index, dtype="object")
    mask = parsed.notna()
    out.loc[mask] = parsed.loc[mask].dt.strftime(fmt_out)
    out.loc[~mask] = series.astype(str).where(~series.isna(), "")
    return out

def clean_tin_preserve(series: pd.Series) -> pd.Series:
    """
    Preserve TIN exactly unless it is empty/NaN or all zeros -> '0'.
    Handles numeric-looking inputs like 12345.0 -> '12345'.
    """
    def norm(v):
        if pd.isna(v):
            return "0"
        if isinstance(v, (int, np.integer)):
            s = str(v)
        elif isinstance(v, float):
            if np.isfinite(v) and v.is_integer():
                s = str(int(v))
            else:
                s = str(v)
        else:
            s = str(v)
        s = s.strip()
        if s == "" or s.lower() in ("nan", "none"):
            return "0"
        if len(s) > 0 and set(s) <= {"0"}:  # zeros-only
            return "0"
        return s
    return series.apply(norm).astype(str)

# Normalize for matching: uppercase, punctuation -> space, collapse spaces
def norm_for_regex(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.upper()
    s = s.str.replace(r"[^\w\s]", " ", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

# ---------- Mapping spec ----------
# kinds: "copy", "concat", "const", "date", "sum", "tin_keep"
mapping_spec = {
    "Number": ("copy", ["application_id"]),
    "Holder Individual": ("copy", ["client_id"]),
    "Holder Entity": ("copy", ["controlling_client_id"]),

    "First Name": ("copy", ["first_name"]),
    "Last Name":  ("copy", ["last_name"]),

    # Street = address_1 + address_2
    "Street": ("concat", ["residential_address_1", "residential_address_2"], {"sep": ", "}),

    "City": ("copy", ["residential_city"]),
    "Country": ("copy", ["country_of_residence"]),
    "Postal Code": ("copy", ["residential_postal_code"]),

    # TIN rule: preserve exactly unless empty/NaN/zeros-only -> '0'
    "TIN 1": ("tin_keep", ["primary_tax_id"]),
    "TIN Issuer 1": ("copy", ["primary_tax_country"]),
    "Tax Residency 1": ("copy", ["primary_tax_country"]),

    # DOB -> dd/mm/YYYY
    "Date of Birth": ("date", ["date_of_birth"], {"fmt": "%d/%m/%Y"}),

    # Business rules
    "Currency": ("const", "USD"),
    "Type": ("copy", ["booking_center"]),
    "Number Type": ("copy", ["client_type"]),

    # Balance rule (choose one)
    "Balance": ("copy", ["cash_balance"]),
    # Or: "Balance": ("sum", ["application_aum", "cash_balance"]),
}

# ---------- Build submission in template order ----------
submission_df = pd.DataFrame(index=raw_data.index)

for tgt in required_columns:
    spec = mapping_spec.get(tgt)
    if not spec:
        submission_df[tgt] = ""  # unmapped targets left blank
        continue

    kind = spec[0]
    sources = spec[1] if len(spec) > 1 else []
    opts = spec[2] if len(spec) > 2 else {}

    if kind == "copy":
        submission_df[tgt] = safe_series(raw_data, sources[0]).astype(str).str.strip()

    elif kind == "concat":
        series_list = [safe_series(raw_data, s).astype(str) for s in sources]
        submission_df[tgt] = concat_with_sep(opts.get("sep", " "), *series_list)

    elif kind == "const":
        value = spec[1] if isinstance(spec[1], str) else opts
        submission_df[tgt] = pd.Series([value] * len(raw_data), index=raw_data.index)

    elif kind == "date":
        fmt = opts.get("fmt", "%d/%m/%Y")
        submission_df[tgt] = parse_date(safe_series(raw_data, sources[0]), fmt_out=fmt)

    elif kind == "sum":
        series_list = [safe_series(raw_data, s, default=0) for s in sources]
        submission_df[tgt] = sum_numeric(*series_list).round(2)

    elif kind == "tin_keep":
        submission_df[tgt] = clean_tin_preserve(safe_series(raw_data, sources[0]))

    else:
        submission_df[tgt] = ""

# Keep exact template order
submission_df = submission_df.reindex(columns=required_columns)

# ---------- Entity name rule via REGEX ----------
# Build a *single* combined regex. We match on normalized (punctuation-stripped) text.
# Each subpattern is designed to match full tokens or *company-style suffixes* at line end.
ENTITY_REGEX_PATTERNS = [
    r'\bSN\s+ISSUER\s+AND\s+BROKER\b$',
    r'\bFUND\s+MANAGER\b$',
    r'\bISSUER\b$',
    r'\bCMA\s+ACC\s+PROVIDER\b$',
    r'\bPVT\s+MKT\s+COUNTERPARTY\b$',
    r'\bCOUNTERPARTY\b$',
    r'\bCLIENT\b$',
    r'\bINTERNATIONAL\s+TRUST\b$',
    r'\bHOLDING\s+LIMITED\b$',

    # Common company suffixes & your examples
    r'\bBANK\s+OF\s+SINGAPORE\s+LTD\b$',
    r'\bHOLDINGS?\s+GLOBAL\s+LTD\b$',
    r'\bSUMMIT\s+INVESTMENTS\s+LTD\b$',
    r'\bPRIMELINK\s+LTD\b$',
    r'\bHOLDINGS?\s+INC\b$',
    r'\bHOLDINGS?\s+LTD\b$',
    r'\bINVESTMENTS?\s+(?:PTE\s+|PTY\s+)?LTD\b$',
    r'\bINNOVATIONS?\s+(?:PTE\s+)?LTD\b$',
    r'\bTRADING\s+(?:PTE\s+)?LTD\b$',
    r'\bSINGAPORE\s+PTE\s+LTD\b$',
    r'\bINC\s+PTE\s+LTD\b$',
    r'\bPTE\s+LTD\b$',       # PTE LTD variations after normalization
    r'\bPTY\s+LTD\b$',
    r'\bLTD\b$'
]

ENTITY_REGEX_COMBINED = r'(?:' + '|'.join(ENTITY_REGEX_PATTERNS) + r')'
# Normalize RAW last_name and test
last_raw_norm = norm_for_regex(safe_series(raw_data, "last_name"))
entity_mask = last_raw_norm.str.contains(ENTITY_REGEX_COMBINED, regex=True, na=False)

# If "Name" exists, set it to "First Name + Last Name" for entity rows
if "Name" in submission_df.columns:
    fn = submission_df.get("First Name", pd.Series("", index=submission_df.index)).astype(str)
    ln = submission_df.get("Last Name",  pd.Series("", index=submission_df.index)).astype(str)
    full_name = concat_with_sep(" ", fn, ln)
    submission_df.loc[entity_mask, "Name"] = full_name.loc[entity_mask]

# Blank First/Last for entity rows
if "First Name" in submission_df.columns:
    submission_df.loc[entity_mask, "First Name"] = ""
if "Last Name" in submission_df.columns:
    submission_df.loc[entity_mask, "Last Name"]  = ""

# ---------- Joint holders counting (dtype-safe) & delete non-primary rows ----------
if "Number" in submission_df.columns:
    # Copy of original Number BEFORE filtering (for counts)
    number_orig = submission_df["Number"].astype(str).str.strip()

    # Identify non-empty Numbers and prefixes
    num_nonempty = number_orig.where(number_orig.ne(""), pd.NA)
    starts_J = number_orig.str.upper().str.startswith("J")
    starts_S = number_orig.str.upper().str.startswith("S")

    # Count identical Number IDs (ignoring blanks)
    group_counts = num_nonempty.groupby(num_nonempty).transform("size")
    group_counts = pd.Series(group_counts, index=submission_df.index).astype("Int64")

    # Build an Int64 series: default 1, J* -> group size, S* -> NA (blank later)
    joint_vals = pd.Series(1, index=submission_df.index, dtype="Int64")
    joint_vals = joint_vals.mask(starts_J & num_nonempty.notna(), group_counts)
    joint_vals = joint_vals.mask(starts_S, pd.NA)

    # Output column name (use template's if present; else create "Nr Joint Holders")
    joint_col = next((c for c in ["Nr Joint Holders", "Joint Holders"] if c in submission_df.columns),
                     "Nr Joint Holders")
    submission_df[joint_col] = joint_vals

    # DELETE rows that are not primary_client_id
    if "Number Type" in submission_df.columns:
        keep_mask = submission_df["Number Type"].astype(str).str.strip().str.lower().eq("primary_client_id")
        submission_df = submission_df.loc[keep_mask].copy()

    # Convert <NA> to "" for Excel display
    submission_df[joint_col] = submission_df[joint_col].astype("object").where(
        submission_df[joint_col].notna(), ""
    )

# ---------- Optional: Is Joint? flag (if present) ----------
if "Is Joint?" in submission_df.columns:
    src_now = submission_df.get("Number", pd.Series("", index=submission_df.index)).astype(str)
    submission_df["Is Joint?"] = np.where(src_now.str.upper().str.startswith("J"), "T", "F")

# ---------- Optional clean-ups ----------
# SG postal code normalize
if "Postal Code" in submission_df.columns:
    submission_df["Postal Code"] = (
        submission_df["Postal Code"].astype(str)
        .str.replace(r"\D", "", regex=True)
        .str.zfill(6)
    )

# ---------- Save ----------
out_path = 'submission_ready.xlsx'
submission_df.to_excel(out_path, index=False)
print(f"Saved: {out_path}")









Saved: submission_ready.xlsx
